In [1]:
import sys
print(sys.executable)   # 예: C:\ProgramData\anaconda3\python.exe
print(sys.path)         # 패키지 탐색 경로(여기에 site-packages가 있어야 import 가능)

c:\Users\USER\anaconda3\python.exe
['c:\\Users\\USER\\aws_etl\\schoolfriends', 'c:\\Users\\USER\\anaconda3\\python311.zip', 'c:\\Users\\USER\\anaconda3\\DLLs', 'c:\\Users\\USER\\anaconda3\\Lib', 'c:\\Users\\USER\\anaconda3', '', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\USER\\anaconda3\\Lib\\site-packages\\Pythonwin']


In [1]:
import awswrangler as wr

In [2]:
df = wr.athena.read_sql_query(
    """
    select a.school_code, a.school_name, b.school_grade, b.school_num, b.class_name, c.psy_name, c.close_yn, f.student_num, e.user_testing_no, e.question_no, e.student_code 
    from school_info a
    inner join school_class b on a.school_code = b.school_code
    inner join psy_class c on b.class_code = c.class_code
    inner join psy_target d on c.class_code = d.class_code and c.psy_code = d.psy_code
    inner join at_user_testing_paper_pn e on d.user_testing_no = e.user_testing_no
    inner join school_student f on d.target_code = f.student_code and d.class_code = f.class_code
    where c.close_yn = 'Y' and e.question_no in ('4','5','6','7','14')
    """
    , database="schoolfriends"
)

In [3]:
df

,school_code,school_name,school_grade,school_num,class_name,psy_name,close_yn,student_num,user_testing_no,question_no,student_code
0,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",Y,"""20""","""557b6318ba464b62be79""",4,AC000120256042D2E003
1,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",Y,"""20""","""557b6318ba464b62be79""",5,AC000120256042D2E003
2,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",Y,"""20""","""557b6318ba464b62be79""",6,AC000120256042D2E003
3,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",Y,"""20""","""557b6318ba464b62be79""",6,AC000120256042D2E009
4,AC0001,학지초등학교,"""6""","""4""",우리 또래 반,"""25년 7월 1회차""",Y,"""20""","""557b6318ba464b62be79""",6,AC000120256042D2E012
...,...,...,...,...,...,...,...,...,...,...,...
5067,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",Y,"""1""",eeaf384f0e31454cb6d0,6,""""""
5068,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",Y,"""1""",eeaf384f0e31454cb6d0,7,""""""
5069,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",Y,"""1""",eeaf384f0e31454cb6d0,14,TC046120256076567012
5070,TC0461,목포부주초등학교,"""6""","""7""","""6학년7반""","""25년 7월 1회차""",Y,"""1""",eeaf384f0e31454cb6d0,14,TC046120256076567021
